In [1]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

import pyspark.sql.functions as F

import sparknlp

spark = sparknlp.start()
# params =>> gpu=False, spark23=False (start with spark 2.3)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

train_CoNLL_file = 'vwb_converted_dataset.conll'
test_CoNLL_file = 'vwb_test_dataset_file'
CSV_file = 'UserStory_For_Testing.csv'

spark

Spark NLP version 3.3.4
Apache Spark version: 3.1.2


In [2]:
with open(CSV_file) as f:
  print (f.read())

#with open('eng.testa') as f:
  c = f.read()

print(c[:1000])

Userstory,Timeline,Goal,Category,Measure
"Show me the revenue across different region, different verticals and across different deparment.",,,"Vertical, region, department",Revenue
Show me revenue for different accounts. Trend of revenue for last 5 years and also the total cost by region.,5 years,,"Accounts, region","Revenue, total cost"
"I would like to visualize profit achieved last year and also the loss incurred, What is the targeted revenue and profit for the next year.","Last Year, Next year",Targeted Revenue,,"Profit, Loss"
"What Is the sales target for next year, Show the number of employees in each department. Also show department wise net profit",Next year,Sales Target,"Employees, Department",Net profit
Plot the number of products sold in a year. Plot number of products in each category. Also the revenue generated across category.,Year,,Category,"Number of product, revenue"
Show me the total premium earned last year and also the average claims incurred per policy type,Last ye

In [3]:
#with open('vwb_dataset_file.conll') as f:
with open(train_CoNLL_file) as f:
  c = f.read()

print(c[:1000])

-DOCSTART- -X- -X- O

Show NNP NNP O
me PRP PRP O
the DT DT O
revenue NN NN B-Goal
across IN IN O
different JJ JJ O
region NN NN B-Cat
, , , O
different JJ JJ O
verticals NNS NNS B-Cat
and CC CC O
across IN IN O
different JJ JJ O
deparment NN NN O
. . . O

Show NNP NNP O
me PRP PRP O
revenue NN NN B-Goal
for IN IN O
different JJ JJ O
accounts NNS NNS B-Cat
. . . O
Trend NNP NNP O
of IN IN O
revenue NN NN B-Goal
for IN IN O
last JJ JJ O
5 CD CD B-Tmln
years NNS NNS I-Tmln
and CC CC O
also RB RB O
the DT DT O
total JJ JJ O
cost NN NN B-Msr
by IN IN O
region NN NN B-Cat
. . . O

I PRP PRP O
would MD MD O
like VB VB O
to TO TO O
visualize NN NN O
profit NN NN B-Goal
achieved VBD VBD O
last JJ JJ B-Tmln
year NN NN I-Tmln
and CC CC O
also RB RB O
the DT DT O
loss NN NN O
incurred VBD VBD O
, , , O
What WP WP O
is VBZ VBZ O
the DT DT O
targeted VBN VBN B-Goal
revenue NN NN I-Goal
and CC CC O
profit NN NN B-Goal
for IN IN O
the DT DT O
next JJ JJ B-Tmln
year NN NN I-Tmln
. . . O

What WP WP O


In [4]:
from sparknlp.training import CoNLL

#training_data = CoNLL().readDataset(spark, 'vwb_dataset_file.conll')
training_data = CoNLL().readDataset(spark, train_CoNLL_file)
training_data.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
training_data.select('sentence').take(100)

[Row(sentence=[Row(annotatorType='document', begin=0, end=97, result='Show me the revenue across different region , different verticals and across different deparment .', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentence=[Row(annotatorType='document', begin=0, end=109, result='Show me revenue for different accounts . Trend of revenue for last 5 years and also the total cost by region .', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentence=[Row(annotatorType='document', begin=0, end=139, result='I would like to visualize profit achieved last year and also the loss incurred , What is the targeted revenue and profit for the next year .', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentence=[Row(annotatorType='document', begin=0, end=126, result='What Is the sales target for next year , Show the number of employees in each department . Also show department wise net profit', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentence=[Row(annotatorType='document', b

In [6]:
training_data.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = tr

In [7]:
training_data.select('sentence').show(5,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|sentence                                                                                                                                                                               |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{document, 0, 97, Show me the revenue across different region , different verticals and across different deparment ., {sentence -> 0}, []}]                                           |
|[{document, 0, 109, Show me revenue for different accounts . Trend of revenue for last 5 years and also the total cost by region ., {sentence -> 0}, []}]                              |
|[{document, 0, 139, I would like to visualize profit achieved last ye

In [8]:
training_data.cache()

DataFrame[text: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentence: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, token: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, pos: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, label: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>]

In [9]:
training_data.count()

15

In [10]:
import pyspark.sql.functions as F

training_data.select(F.explode(F.arrays_zip(training_data.token.result, 
                                            training_data.pos.result, 
                                            training_data.label.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("token"),
                     F.expr("cols['1']").alias("pos"),
                     F.expr("cols['2']").alias("ner_label")).show(truncate=False)

+---------+---+---------+
|token    |pos|ner_label|
+---------+---+---------+
|Show     |NNP|O        |
|me       |PRP|O        |
|the      |DT |O        |
|revenue  |NN |B-Goal   |
|across   |IN |O        |
|different|JJ |O        |
|region   |NN |B-Cat    |
|,        |,  |O        |
|different|JJ |O        |
|verticals|NNS|B-Cat    |
|and      |CC |O        |
|across   |IN |O        |
|different|JJ |O        |
|deparment|NN |O        |
|.        |.  |O        |
|Show     |NNP|O        |
|me       |PRP|O        |
|revenue  |NN |B-Goal   |
|for      |IN |O        |
|different|JJ |O        |
+---------+---+---------+
only showing top 20 rows



In [11]:
training_data.select(F.explode(F.arrays_zip(training_data.token.result,
                                            training_data.label.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("token"),
                     F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)

+------------+-----+
|ground_truth|count|
+------------+-----+
|O           |285  |
|B-Cat       |21   |
|B-Goal      |20   |
|B-Msr       |6    |
|I-Tmln      |3    |
|B-Tmln      |3    |
|I-Cat       |2    |
|I-Goal      |2    |
+------------+-----+



In [12]:
# we use BERT Tiny
#from flair.embeddings import BertEmbeddings

bert_annotator = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
  #.setBatchSize(8)

small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


In [13]:
!mkdir ner_logs

A subdirectory or file ner_logs already exists.


In [14]:
#You can also set learning rate ( setLr ), learning rate decay coefficient ( setPo ), setBatchSize and setDropout rate. 
#Please see the official APIs for the entire list.
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "bert"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(100)\
  .setLr(0.001)\
  .setPo(0.005)\
  .setBatchSize(32)\
  .setVerbose(1)\
  .setValidationSplit(0.2)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setIncludeConfidence(True)\
  .setOutputLogsPath('ner_logs')
  #.setTestDataset("VW_test_withEmbeds.parquet")

pipeline = Pipeline(
    stages = [
    bert_annotator,
    nerTagger
  ])

### Fitting

In [15]:
%%time

ner_model = pipeline.fit(training_data)

Wall time: 22.3 s


In [16]:
import os

log_files = os.listdir("ner_logs")
log_files

['NerDLApproach_225fd5c0935a.log',
 'NerDLApproach_271ca6f6f6b3.log',
 'NerDLApproach_3b53d7e372da.log',
 'NerDLApproach_3db26ce7e2a8.log',
 'NerDLApproach_4c38d769ccc2.log',
 'NerDLApproach_7ee50629449e.log',
 'NerDLApproach_9d35030317ab.log',
 'NerDLApproach_c111401a504b.log',
 'NerDLApproach_e6bbed0f557e.log']

In [17]:
with open("ner_logs/"+log_files[0]) as log_file:
    print(log_file.read())

Name of the selected graph: ner-dl/blstm_10_128_128_120.pb
Training started - total epochs: 100 - lr: 0.001 - batch size: 32 - labels: 3 - chars: 38 - training examples: 12


Epoch 1/100 started, lr: 0.001, dataset size: 12


Epoch 1/100 - 2.39s - loss: 69.975815 - batches: 2
Quality on validation dataset (20.0%), validation examples = 2
time to finish evaluation: 0.33s
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Cat	 0	 0	 1	 0.0	 0.0	 0.0
I-Goal	 0	 0	 3	 0.0	 0.0	 0.0
tp: 0 fp: 0 fn: 4 labels: 2
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0


Epoch 2/100 started, lr: 9.950249E-4, dataset size: 12


Epoch 2/100 - 0.24s - loss: 38.293617 - batches: 2
Quality on validation dataset (20.0%), validation examples = 2
time to finish evaluation: 0.02s
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Cat	 0	 0	 1	 0.0	 0.0	 0.0
I-Goal	 0	 0	 3	 0.0	 0.0	 0.0
tp: 0 fp: 0 fn: 4 labels: 2
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0



In [18]:
from sparknlp.training import CoNLL

test_data = CoNLL().readDataset(spark, train_CoNLL_file)

test_data = bert_annotator.transform(test_data)

test_data.show(10,truncate = False)

+--------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
#test_data.select('sentence').take(20)

In [20]:
predictions = ner_model.transform(test_data.select("sentence", "token", "label"))
predictions.show(3,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
predictions.select('token.result','label.result','ner.result').show(truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                        |result                                                                                                                   |result                                                                                                                   |
+-----------------------------------------------------------------------------------------------------------------------------

### Test set evaluation

In [22]:
import pyspark.sql.functions as F

predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
  .select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth"),
        F.expr("cols['2']").alias("prediction")).show(truncate=False)

+---------+------------+----------+
|token    |ground_truth|prediction|
+---------+------------+----------+
|Show     |O           |O         |
|me       |O           |O         |
|the      |O           |O         |
|revenue  |B-Goal      |B-Goal    |
|across   |O           |O         |
|different|O           |O         |
|region   |B-Cat       |B-Cat     |
|,        |O           |O         |
|different|O           |O         |
|verticals|B-Cat       |O         |
|and      |O           |O         |
|across   |O           |O         |
|different|O           |O         |
|deparment|O           |B-Cat     |
|.        |O           |O         |
|Show     |O           |O         |
|me       |O           |O         |
|revenue  |B-Goal      |B-Goal    |
|for      |O           |O         |
|different|O           |O         |
+---------+------------+----------+
only showing top 20 rows



In [23]:
from sklearn.metrics import classification_report

preds_df = predictions.select(F.explode(F.arrays_zip(predictions.token.result,
                                                     predictions.label.result,
                                                     predictions.ner.result)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()

print (classification_report(preds_df['ground_truth'], preds_df['prediction']))

              precision    recall  f1-score   support

       B-Cat       0.95      0.95      0.95        21
      B-Goal       1.00      1.00      1.00        20
       B-Msr       1.00      1.00      1.00         6
      B-Tmln       1.00      1.00      1.00         3
       I-Cat       1.00      1.00      1.00         2
      I-Goal       1.00      1.00      1.00         2
      I-Tmln       1.00      1.00      1.00         3
           O       1.00      1.00      1.00       285

    accuracy                           0.99       342
   macro avg       0.99      0.99      0.99       342
weighted avg       0.99      0.99      0.99       342



In [24]:
import conll_eval

metrics = conll_eval.evaluate(preds_df['ground_truth'].values, preds_df['prediction'].values)

processed 342 tokens with 50 phrases; found: 50 phrases; correct: 49.
accuracy:  98.25%; (non-O)
accuracy:  99.42%; precision:  98.00%; recall:  98.00%; FB1:  98.00
              Cat: precision:  95.24%; recall:  95.24%; FB1:  95.24  21
             Goal: precision: 100.00%; recall: 100.00%; FB1: 100.00  20
              Msr: precision: 100.00%; recall: 100.00%; FB1: 100.00  6
             Tmln: precision: 100.00%; recall: 100.00%; FB1: 100.00  3


In [25]:
# micro, macro, avg
metrics[0]

(98.0, 98.0, 98.0)

In [26]:
import pandas as pd
pd.DataFrame(metrics[1], columns=['entity','precision','recall','f1','support'])

,entity,precision,recall,f1,support
0,Cat,95.238095,95.238095,95.238095,21
1,Goal,100.000000,100.000000,100.000000,20
2,Msr,100.000000,100.000000,100.000000,6
3,Tmln,100.000000,100.000000,100.000000,3


### Splitting Dataset Into Train and Test Set

In [27]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, train_CoNLL_file) #Training Data set

(training_data, test_data) = conll_data.randomSplit([0.7, 0.3], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 10
Test Dataset Count: 5


In [28]:
!mkdir ner_logs_best

A subdirectory or file ner_logs_best already exists.


In [29]:
# let's transform and save our test dataset for evaluation
#test_data.write.parquet("VW_test_withEmbeds.parquet")
bert_annotator.transform(test_data).write.parquet('ner_dl_test.parquet')

In [30]:
%%time
nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "bert"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(100)\
      .setLr(0.001)\
      .setPo(0.005)\
      .setBatchSize(32)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset("ner_dl_test.parquet")\
      .setOutputLogsPath('ner_logs_best')

pipeline = Pipeline(
    stages = [
    bert_annotator,
    nerTagger
  ])

ner_model = pipeline.fit(training_data)

Wall time: 26 s


In [31]:
log_files = os.listdir("ner_logs_best/")
log_files

['NerDLApproach_1b320d1c7b67.log',
 'NerDLApproach_5690f906ffe9.log',
 'NerDLApproach_672e9aec2464.log',
 'NerDLApproach_9330e957d510.log',
 'NerDLApproach_99aacd61afae.log',
 'NerDLApproach_dfb50c9aea44.log',
 'NerDLApproach_ffa9c3de3877.log']

In [32]:
with open("ner_logs_best/"+log_files[0]) as log_file:
    print(log_file.read())

Name of the selected graph: ner-dl/blstm_10_128_128_120.pb
Training started - total epochs: 100 - lr: 0.001 - batch size: 32 - labels: 7 - chars: 35 - training examples: 8


Epoch 1/100 started, lr: 0.001, dataset size: 8


Epoch 1/100 - 2.24s - loss: 73.04648 - batches: 2
Quality on validation dataset (20.0%), validation examples = 1
time to finish evaluation: 0.40s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-Cat	 0	 0	 2	 0.0	 0.0	 0.0
B-Goal	 0	 0	 1	 0.0	 0.0	 0.0
tp: 0 fp: 0 fn: 3 labels: 2
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Quality on test dataset: 
time to finish evaluation: 0.20s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-Goal	 0	 0	 6	 0.0	 0.0	 0.0
B-Msr	 0	 0	 2	 0.0	 0.0	 0.0
B-Cat	 0	 0	 11	 0.0	 0.0	 0.0
I-Cat	 0	 0	 2	 0.0	 0.0	 0.0
tp: 0 fp: 0 fn: 21 labels: 4
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0


Epoch 2/100 started, lr: 9.950249E-4, dataset size: 8


Epoch 2/100 - 0.44s - loss: 55.

In [33]:
test_data = bert_annotator.transform(test_data.select("sentence", "token", "label"))

predictions = ner_model.transform(test_data.select("sentence", "token", "label"))

from sklearn.metrics import classification_report

preds_df = predictions.select(F.explode(F.arrays_zip(predictions.token.result,
                                                     predictions.label.result,
                                                     predictions.ner.result)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()

print(classification_report(preds_df['ground_truth'], preds_df['prediction'], digits=4))

              precision    recall  f1-score   support

       B-Cat     0.2500    0.0909    0.1333        11
      B-Goal     0.6250    0.8333    0.7143         6
       B-Msr     1.0000    0.5000    0.6667         2
      B-Tmln     0.0000    0.0000    0.0000         0
       I-Cat     0.0000    0.0000    0.0000         2
      I-Tmln     0.0000    0.0000    0.0000         0
           O     0.8673    0.9140    0.8901        93

    accuracy                         0.8070       114
   macro avg     0.3918    0.3340    0.3435       114
weighted avg     0.7821    0.8070    0.7883       114



C:\Users\65361\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\65361\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\65361\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\65361\Anaconda3\envs\nlp\li

### Saving the trained model

In [34]:
ner_model.stages

[BERT_EMBEDDINGS_68e8f02f4bcd, NerDLModel_6bd52491dd75]

In [35]:
ner_model.stages[1].write().overwrite().save('Vwb_NER_BERT_e5_b32_16122022')

### Prediction Pipeline

In [36]:
from pyspark.ml import Pipeline

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

bert = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
 .setCaseSensitive(True)

loaded_ner_model = NerDLModel.load("Vmb_NER_BERT_e5_b32_16122022")\
 .setInputCols(["sentence", "token", "bert"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        bert,
        loaded_ner_model,
        converter])

small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


In [37]:
empty_data = spark.createDataFrame([['']]).toDF('text')
empty_data.show()

+----+
|text|
+----+
|    |
+----+



In [38]:
prediction_model = ner_prediction_pipeline.fit(empty_data)

In [39]:
#text = "I would like to visualize profit achieved last year and also the loss incurred, What is the targeted revenue and profit for the next year."
#text = 'Show me the revenue across different region, different verticals and across different deparment.'
text = 'I would like to visualize profit achieved last year and also the loss incurred, What is the targeted revenue and profit for the next year.'

sample_data = spark.createDataFrame([[text]]).toDF("text")
sample_data.show()

+--------------------+
|                text|
+--------------------+
|I would like to v...|
+--------------------+



In [40]:
preds = prediction_model.transform(sample_data)

preds.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [41]:
preds.select('ner_span.result').take(1)

[Row(result=['profit', 'last year', 'targeted revenue', 'profit', 'next year'])]

In [42]:
preds.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities")) \
.select(F.expr("entities['0']").alias("chunk"),
        F.expr("entities['1'].entity").alias("entity")).show(truncate=False)

+----------------+------+
|chunk           |entity|
+----------------+------+
|profit          |Goal  |
|last year       |Tmln  |
|targeted revenue|Goal  |
|profit          |Goal  |
|next year       |Tmln  |
+----------------+------+



### Pretrained Pipelines

In [43]:
from sparknlp.pretrained import PretrainedPipeline

pretrained_pipeline = PretrainedPipeline('recognize_entities_dl', lang='en')

#onto_recognize_entities_sm
#explain_document_dl

recognize_entities_dl download started this may take some time.
Approx size to download 160.1 MB
[OK!]


In [44]:
#text = "The Mona Lisa is a 16th century oil painting created by Leonardo. It's held at the Louvre in Paris."

#text = "I would like to visualize profit achieved last year and also the loss incurred, What is the targeted revenue and profit for the next year."
text = 'Show me the revenue across different region, different verticals and across different deparment.'

result = pretrained_pipeline.annotate(text)

list(zip(result['token'], result['ner']))


[('Show', 'O'),
 ('me', 'O'),
 ('the', 'O'),
 ('revenue', 'O'),
 ('across', 'O'),
 ('different', 'O'),
 ('region', 'O'),
 (',', 'O'),
 ('different', 'O'),
 ('verticals', 'O'),
 ('and', 'O'),
 ('across', 'O'),
 ('different', 'O'),
 ('deparment', 'O'),
 ('.', 'O')]

In [45]:
result.keys()

dict_keys(['entities', 'document', 'token', 'ner', 'embeddings', 'sentence'])

In [46]:
# list(zip(result2['token'],  result2['checked'], result2['pos'], result2['ner'],  result2['lemma'],  result2['stem']))
#list(zip(result2['token'], result2['spell'], result2['pos'], result2['lemmas'],  result2['stems']))
#list(zip(result['token'], result['spell'], result['pos'], result['lemmas'],  result['stems']))
list(zip(result['document'], result['token'], result['ner'],  result['embeddings'],  result['entities']))

[]